<a href="https://colab.research.google.com/github/heirynhr/legalduel-1a/blob/main/LegalDuel1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dateparser

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [ ]:

!pip install nltk


In [ ]:
# importing libraries here

from transformers import BartTokenizer, BartForConditionalGeneration
import torch
from PyPDF2 import PdfReader
from google.colab import files
import spacy
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import dateparser
from transformers import pipeline
nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# this is how we are able to upload the different chronologies into the document
# run this cell to add new docs

uploaded = files.upload()

Saving Chron.B.1.docx to Chron.B.1.docx


In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


---- Here is where we will be experimenting with different date recognition tools

In [42]:
from docx import Document

file_path = uploaded['Chron.B.1.docx']

# Open the file in binary mode ('rb') for reading
with open('Chron.B.1.docx', 'rb') as f:
    doc = Document(f)

conversation = []
for line in doc.paragraphs:
  conversation.append(line.text)

print(len(conversation))

104


In [56]:
def validate_dates(date):
  date_obj = None
  try:
    # Corrected the format string for the date with the year included
    date_obj = datetime.strptime(date, "%B %d, %Y")
  except ValueError:
    try:
      # If no year, try to format the date with just month and day
      date_obj = datetime.strptime(date, "%B %d")
    except ValueError:
      # If it's not a valid date, the error will be handled here
      date_obj = None
  return date_obj

In [58]:
nlp = spacy.load("en_core_web_sm")

tokens = []
for item in conversation:
  line = nlp(item)
  dates = [ent for ent in line.ents if ent.label_ in ("DATE")]
  for date in dates:
    date_str = date.text.strip()

    validated_date = validate_dates(date_str)

    token = (validated_date, item)
    tokens.append(token)


print(tokens[10])

(datetime.datetime(2023, 11, 25, 0, 0), 'Rachel Adams: Yes, I contacted customer support again on November 25, 2023, and they sent a replacement unit, which arrived on December 1, 2023. However, the replacement unit had similar issues and did not resolve the problem.')


In [61]:
def summarize_date_tokens(tokens):
  summaries = []
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  for date_obj, context in tokens:

    summary = summarizer(context, max_length=40, min_length=5, do_sample=False)

    # Formatting to exclude extra built in text
    summary_strip = summary[0]
    summary_text = summary_strip['summary_text']

    event = f"{date_str}: {summary_text}"

    summaries.append(event)

  return summaries

summaries = summarize_date_tokens(tokens)
summaries

Your max_length is set to 40, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 40, but your input_length is only 35. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 40, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 40, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_

["March 1, 2024: CNN will be broadcasting a live special on CNN.com on August 22, 2024. Click here for more information about CNN's upcoming special.",
 'March 1, 2024: Rachel Adams is a marketing manager in Denver, Colorado. She lives with her husband and three children.',
 'March 1, 2024: Rachel Adams first heard about the SafeHome Security System in September 2023. It was advertised as a comprehensive home security solution with advanced features.',
 'March 1, 2024: Rachel Adams purchased the book on October 10, 2023. She bought it online from SafeHome.',
 "March 1, 2024: Initially, the system seemed to work well. We were impressed with the app's interface and the ability to monitor our home remotely.",
 'March 1, 2024: The first issues began in November 2023. On November 5, 2023, the system started to malfunction.',
 'March 1, 2024: The first issues began in November 2023. On November 5, 2023, the system started to malfunction.',
 'March 1, 2024: The first issues began in November 

**future steps**

document layout analysis: detecting layout of the document
- sentence segmentation: ssplitting text into sentences -> could replace context window or add to it
  - lots of different models for sentence segmentation.
  - maybe split by periods and then remove the "erma wiggins" after
- we should add "no year detected" to where we don't have years so that we aren't misconstruing information
  - adding some precautionary text